In [1]:
from qiskit import IBMQ
import qiskit
qiskit.__qiskit_version__

{'qiskit-terra': '0.10.0',
 'qiskit-aer': '0.3.2',
 'qiskit-ignis': '0.2.0',
 'qiskit-ibmq-provider': '0.3.3',
 'qiskit-aqua': '0.6.1',
 'qiskit': '0.13.0'}

In [2]:
provider = IBMQ.load_account()
#change the provider based on your Qiskit account
#if you do not have access to the twenty qubit machine, use the publically available backend instead
provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
provider.backends()

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q-ncsu', group='nc-state', project='on-boarding')>,
 <IBMQBackend('ibmq_20_tokyo') from IBMQ(hub='ibm-q-ncsu', group='nc-state', project='on-boarding')>,
 <IBMQBackend('ibmq_poughkeepsie') from IBMQ(hub='ibm-q-ncsu', group='nc-state', project='on-boarding')>,
 <IBMQBackend('ibmq_boeblingen') from IBMQ(hub='ibm-q-ncsu', group='nc-state', project='on-boarding')>,
 <IBMQBackend('ibmq_vigo') from IBMQ(hub='ibm-q-ncsu', group='nc-state', project='on-boarding')>,
 <IBMQBackend('ibmq_ourense') from IBMQ(hub='ibm-q-ncsu', group='nc-state', project='on-boarding')>,
 <IBMQBackend('ibmq_valencia') from IBMQ(hub='ibm-q-ncsu', group='nc-state', project='on-boarding')>,
 <IBMQBackend('ibmq_london') from IBMQ(hub='ibm-q-ncsu', group='nc-state', project='on-boarding')>,
 <IBMQBackend('ibmq_burlington') from IBMQ(hub='ibm-q-ncsu', group='nc-state', project='on-boarding')>,
 <IBMQBackend('ibmq_almaden') from IBMQ(hub='ibm-q-ncsu', group='nc-state

In [3]:
from qiskit import *
from qiskit.compiler import transpile
from qiskit.visualization import plot_histogram
from qiskit.tools.monitor import job_monitor, backend_monitor, backend_overview
from qiskit.providers.aer import noise
import math

import numpy as np

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import Aer, execute

from qiskit.providers.aer import UnitarySimulator
import qiskit.ignis.verification.tomography.basis.circuits

In [4]:
# Needed for functions
import numpy as np
import time
from copy import deepcopy

# Import Qiskit classes
import qiskit 
from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister, Aer
from qiskit.quantum_info import state_fidelity
from qiskit.providers.aer import noise

# Tomography functions
from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter
import qiskit.ignis.mitigation.measurement as mc

In [5]:
#We used ibmq_20_tokyo for our experiment, however, that backend has retired. So we can use the publically available ibmqx2 backend instead
device = provider.get_backend('ibmqx2')
#if you have access to the twenty qubit machine, use this 20 qubit machine instead
#device = provider.get_backend('ibmq_boeblingen')
backend_monitor(device)
device

ibmq_boeblingen
Configuration
-------------
    n_qubits: 20
    operational: True
    status_msg: active
    pending_jobs: 4
    backend_version: 1.0.3
    basis_gates: ['u1', 'u2', 'u3', 'cx', 'id']
    local: False
    simulator: False
    u_channel_lo: [[{'q': 1, 'scale': [1.0, 0.0]}], [{'q': 0, 'scale': [1.0, 0.0]}], [{'q': 2, 'scale': [1.0, 0.0]}], [{'q': 6, 'scale': [1.0, 0.0]}], [{'q': 1, 'scale': [1.0, 0.0]}], [{'q': 3, 'scale': [1.0, 0.0]}], [{'q': 2, 'scale': [1.0, 0.0]}], [{'q': 4, 'scale': [1.0, 0.0]}], [{'q': 8, 'scale': [1.0, 0.0]}], [{'q': 3, 'scale': [1.0, 0.0]}], [{'q': 6, 'scale': [1.0, 0.0]}], [{'q': 10, 'scale': [1.0, 0.0]}], [{'q': 1, 'scale': [1.0, 0.0]}], [{'q': 5, 'scale': [1.0, 0.0]}], [{'q': 7, 'scale': [1.0, 0.0]}], [{'q': 6, 'scale': [1.0, 0.0]}], [{'q': 8, 'scale': [1.0, 0.0]}], [{'q': 12, 'scale': [1.0, 0.0]}], [{'q': 3, 'scale': [1.0, 0.0]}], [{'q': 7, 'scale': [1.0, 0.0]}], [{'q': 9, 'scale': [1.0, 0.0]}], [{'q': 8, 'scale': [1.0, 0.0]}], [{'q': 14, 'sc

<IBMQBackend('ibmq_boeblingen') from IBMQ(hub='ibm-q-ncsu', group='nc-state', project='on-boarding')>

In [6]:
#entangle3bit:
qr = QuantumRegister(4)
circ = QuantumCircuit(qr)
#assertion circuit for three qubit entanglement, because the measurement in state_tomography_circuit and the 
#measurement of ancilla qubit in our assertion function will cause conflict, we manually insert the assertion circuits
circ.cx(qr[0], qr[3])
circ.cx(qr[1], qr[3])
circ.cx(qr[2], qr[3])
circ.cx(qr[1], qr[3])

tomocircuits=state_tomography_circuits(circ, qr, meas_labels='Pauli', meas_basis='Pauli')
trans_qc = transpile(tomocircuits, device, optimization_level = 3)

In [7]:
jobSim = qiskit.execute(circ, Aer.get_backend('statevector_simulator'))
psi_circ = jobSim.result().get_statevector(circ)
print(psi_circ)

[1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]


In [8]:
shots = 8192
#execute the job on real device
job = execute(trans_qc, backend=device, shots=shots)
job_monitor(job)
res = job.result()

Job Status: job has successfully run


In [9]:
res.job_id

'5de805dde2b9c600111b3bfe'

In [10]:
tomo_state = StateTomographyFitter(job.result(), tomocircuits)

In [11]:
rho_bell = tomo_state.fit()
F_state = state_fidelity(psi_circ, rho_bell)
print('Assertion Circuit Fidelity =', F_state)

Assertion Circuit Fidelity = 0.9067804340558016
